In [2]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from collections import Counter

In [3]:
#home='./Assignment-files'
dest_path=r'C:\Users\tanya\Desktop\Tag Files'
home=r'C:\Users\tanya\Desktop\Assignment-Files'
result=r'C:\Users\tanya\Desktop\2020-21\AI\word_tag.txt'


In [4]:
# creating word and tag dictionaries #


def preprocess(dest_path):
    for folder in os.listdir(dest_path):
        local_dest=dest_path+'/'+folder
        
        for file in os.listdir(local_dest):
            make_text(local_dest+"/"+file)
            
            
def make_text(dest):
    with open(dest,errors='ignore') as fp:
        for line in fp:
            
            word_tag_cnt(line.strip().split(' '))

def word_tag_cnt(line):
    global totalWordTagLen
    for words in line:
        
        word=words.split('_')[0]
        splitted_tags=words.split('_')[-1].split('-')
        totalWordTagLen+=len(splitted_tags)
        if word in freq_of_word:
            freq_of_word[word]+=1
        else:
            freq_of_word[word]=1
        if word not in freq_of_word_with_tag:
            freq_of_word_with_tag[word]={}
        for tag in splitted_tags:
            if tag in freq_of_tag:
                freq_of_tag[tag]+=1
            else:
                freq_of_tag[tag]=1
            if tag in freq_of_word_with_tag[word]:
                freq_of_word_with_tag[word][tag]+=1
            else:
                freq_of_word_with_tag[word][tag]=1
   

In [5]:
data = ['Train-corpus']
freq_of_word={}
freq_of_tag={}
freq_of_word_with_tag={} #nested dictionary to store word tag pair frequency

totalWordTagLen=0
for i in data:
    #source_path=os.path.join(home,i)
    path=os.path.join(dest_path,i)
   
    preprocess(path)

In [ ]:
# freq of word dictionary
# freq of tag dictionary
# freq of word tag dictionary

In [6]:
#print('Top 10 frequently used words are:')
k = Counter(freq_of_word) 
high = k.most_common(10) 
#for i in high:
    #print("\t",i[0]," :",i[1]," ")

#to get the top 10 frequently used tags
#print('Top 10 frequently used tags are:')
k = Counter(freq_of_tag) 
high = k.most_common(10) 
#for i in high:
   # print("\t",i[0]," :",i[1])

In [7]:
prob_of_tag={}
list_of_words=freq_of_word.keys()
list_of_tags=freq_of_tag.keys()

#P(tag)

for key, value in sorted(freq_of_tag.items()):
    prob_of_tag[key]=value/totalWordTagLen


In [ ]:
# list_of_words
# list_of_tags

In [13]:
#P(word|tag)
prob_of_word_given_tag={} 
for tag in list_of_tags:
    prob_of_word_given_tag[tag]={}
    for word in list_of_words:
        if tag in freq_of_word_with_tag[word]:
            prob_of_word_given_tag[tag][word]=freq_of_word_with_tag[word][tag]/freq_of_tag[tag]
        else:
            prob_of_word_given_tag[tag][word]=0

#P(tag|word) using Bayes rule
prob_of_tag_given_word={}
with open(result,'w') as fp:
    for word in list_of_words:
        fp.write(word+"_")
        maxi=0
        prob_of_tag_given_word[word]={}
        for tag in list_of_tags:
            if tag in freq_of_word_with_tag[word]:
                prob_of_tag_given_word[word][tag]=prob_of_word_given_tag[tag][word]*prob_of_tag[tag]
                if prob_of_tag_given_word[word][tag]>maxi:
                    maxi=prob_of_tag_given_word[word][tag]
                    max_tag=tag
        fp.write(max_tag)
        fp.write("\n")


In [14]:
trained_word_tag={} #using_bayes_rule
with open(result,'r') as fp:
    for line in fp:
        line = line.replace('\n', '')
        (key, val) = line.split('_')
        trained_word_tag[key] = val



In [9]:

def hmm():

#  compute emission probability = P(word|tag)                    #
# creating t x w emission matrix of tags*words, w=no.of words and t= no of tags 
#                        Matrix(i, j) represents P(j|i)

    global emission_matrix
    for i, tag in enumerate(list_of_tags):
        for j, word in enumerate(list_of_words): 
            if tag in prob_of_word_given_tag and word in prob_of_word_given_tag[tag]:
                emission_matrix[i, j] = prob_of_word_given_tag[tag][word]
    #emission_matrix[:,-1]=1e-12
    global emission_matrix_df
    emission_matrix_df.to_csv(r'emission_matrix.csv', header=True, index=True)
    
    transitionp()
    global transition_matrix
    for i, tag1 in enumerate(list_of_tags):
        for j, tag2 in enumerate(list_of_tags): 
            if tag1 in freq_of_tag2_given_tag1 and tag2 in freq_of_tag2_given_tag1[tag1]:
                transition_matrix[i, j] = freq_of_tag2_given_tag1[tag1][tag2]/outgoing_tag_count[tag1]

    global transition_matrix_df
    transition_matrix_df.to_csv(r'transition_matrix.csv', header=True, index=True)
    
    global tag_prior_probabilities
    for i, tag in enumerate(freq_of_tag.keys()):
        tag_prior_probabilities[i]=freq_of_tag[tag]/totalWordTagLen

    global tag_prior_probabilities_df
    tag_prior_probabilities_df.to_csv(r'tag_prior_probabilities.csv',header=True, index=True, index_label='tag')



In [10]:
def preprocesss(dest_path):
    for folder in os.listdir(dest_path):
        local_dest=dest_path+'/'+folder
        for file in os.listdir(local_dest):
            make_textt(local_dest+"/"+file)
            
            
def make_textt(dest):
    global freq_of_tag2_given_tag1
    global outgoing_tag_count
    with open(dest,errors='ignore') as fp:
        for line in fp:
            tag=['start']
            for words in line.strip().split(' '):
                splitted_tags=words.split('_')[-1].split('-')
                for tag1 in tag:
                    if tag1 not in freq_of_tag2_given_tag1:
                        freq_of_tag2_given_tag1[tag1]={}
                    for tag2 in splitted_tags:
                        if tag2 in freq_of_tag2_given_tag1[tag1]:
                            freq_of_tag2_given_tag1[tag1][tag2]+=1
                        else:
                            freq_of_tag2_given_tag1[tag1][tag2]=1
                        if tag1 in outgoing_tag_count:
                            outgoing_tag_count[tag1]+=1
                        else:
                            outgoing_tag_count[tag1]=1
                tag=splitted_tags

            
            
def transitionp():
    data=['Train-corpus']
    for i in data:
        path=os.path.join(dest_path,i)
        preprocesss(path)

In [15]:
emission_matrix = np.zeros((len(list_of_tags), len(list_of_words)), dtype='float32')
emission_matrix_df = pd.DataFrame(emission_matrix, columns = list(list_of_words), index=list_of_tags)
freq_of_tag2_given_tag1={}
outgoing_tag_count={}
transition_matrix = np.zeros((len(list_of_tags), len(list_of_tags)), dtype='float32')
transition_matrix_df = pd.DataFrame(transition_matrix, index = list_of_tags, columns=list_of_tags)
tag_prior_probabilities = np.zeros((len(list_of_tags)), dtype='float32')
tag_prior_probabilities_df = pd.DataFrame(tag_prior_probabilities, index = list_of_tags, columns=['P(tag)'])
hmm()

In [144]:
# TESTING USING BAYES RULE #

In [78]:
actualTags=[]
predictedTags=[]
correctCount=0
testFileLen=0
data1 = ['Test-corpus']
for i in data1:
    #source_path=os.path.join(home,i)
    path=os.path.join(dest_path,i)
   
    preprocess2(path)

            
            
            


In [77]:
def make_text2(dest):
    with open(dest,errors='ignore') as fp:
        for line in fp:
            for item in line.strip().split(' '):
                global testFileLen
                testFileLen+=1
                wordtag=item.split('_')
                givenword=wordtag[0]
                giventag=wordtag[-1]
                actualTags.append(giventag)
                global correctCount   
                if givenword in trained_word_tag:
                    predictedTags.append(trained_word_tag[givenword])
                    if giventag==trained_word_tag[givenword]:
                        correctCount=correctCount+1
                elif giventag=='NN1':
                    predictedTags.append('NN1')
                    correctCount=correctCount+1
                else:
                    predictedTags.append('NN1')
def preprocess2(dest_path):
    for folder in os.listdir(dest_path):
        local_dest=dest_path+'/'+folder
        for file in os.listdir(local_dest):
            make_text2(local_dest+"/"+file)

In [80]:
print("Accuracy of the system is : " + str(correctCount*100/testFileLen))


Accuracy of the system is : 89.243479058913


In [81]:
data = {'y_Actual':   actualTags ,
        'y_Predicted':predictedTags 
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])

confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'])

confusion_matrix.to_csv('cmatrix.csv')


In [83]:

confusion_matrix=pd.read_csv('cmatrix.csv',index_col=0,header=0)

confusion_matrix

,VHN,AJ0,AJC,AJS,AT0,AV0,AVP,AVQ,CJC,CJS,...,VHZ,VM0,VVB,VVD,VVG,VVI,VVN,VVZ,XX0,ZZ0
AJ0,0,246318,0,0,0,3636,328,0,0,0,...,0,1,48,1480,7304,327,10790,0,0,11
AJC,0,0,6694,5,0,1073,0,0,0,0,...,0,0,6,0,0,4,0,0,0,0
AJS,0,0,0,3751,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AT0,0,0,0,0,342544,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AV0,0,6958,1372,471,429,159262,0,0,14,2682,...,0,0,1,0,12,19,24,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VVI,0,1409,32,0,8,50,27,0,0,0,...,0,17,4072,159,0,61819,1013,0,0,0
VVN,0,4709,0,0,0,0,0,0,0,4,...,0,6,55,19159,0,1409,71973,0,0,0
VVZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,23351,0,0
XX0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24927,0


In [84]:
no_of_times_predicted=[]
F1_score=[]
print("Tag"+'\t\t'+"precision"+'\t\t\t\t'+"recall"+'\t\t\t\t'+"F-score")
for tag in confusion_matrix.index.values.tolist():
    TP=confusion_matrix.at[tag,tag]
    row_list=confusion_matrix.loc[tag,:].values.tolist()
    FN=sum(row_list)-TP
    column_list=confusion_matrix.loc[:,tag].values.tolist()
    FP=sum(column_list)-TP
    if TP+FP==0:
        p=0
    else:
        p=TP/(TP+FP)
    if TP+FN==0:
        r=0
    else:
        r=TP/(TP+FN)
    if p+r==0:
        Fscore=0
    else:
        Fscore = 2 * (p * r) / (p+r)
    no_of_times_predicted.append(sum(column_list))
    F1_score.append(Fscore)
    print(tag+'\t\t'+str(p)+'\t\t'+str(r)+'\t\t'+str(Fscore))

print("Macro F1 score: "+str(sum(F1_score)/len(F1_score)))
num=0
den=0
for (n,f) in zip(no_of_times_predicted,F1_score):
    num+=(n*f)
    den+=n
print("Weighted F1 score: "+str(num/den))

Tag		precision				recall				F-score
AJ0		0.8529873152082446		0.8402169463774048		0.8465539728936936
AJC		0.810411622276029		0.8450953162479485		0.827390148940115
AJS		0.882173095014111		0.983739837398374		0.9301921884686919
AT0		0.993330317476888		0.9999503738020382		0.9966293524196072
AV0		0.9264481751654974		0.871241089940317		0.8979969270238649
AVP		0.8260249796361662		0.5997979150750425		0.6949643780071671
AVQ		0.7843118656928768		0.7298323036187114		0.7560919855529649
CJC		0.9988059345055076		0.9999925282246315		0.9993988791523077
CJS		0.749989211272063		0.8718999280900379		0.8063628068113275
CJT		0.7263005211918576		0.9944124676024101		0.8394686367834057
CRD		0.9342321037095855		0.9481272555271364		0.9411283941815274
DPS		0.9464818763326226		0.9998613878541107		0.9724396511774867
DT0		0.9567155090883179		0.8071395000601153		0.8755854349707728
DTQ		0.9999573014517507		0.9998292276821927		0.9998932604658113
EX0		0.8224518145511374		0.9998944479628457		0.9025342987804877
ITJ		0.9759

In [16]:


#start_tag=pd.read_csv('tag_prior_probabilities.csv',index_col=0,header=0)
emission_df=pd.read_csv('emission_matrix.csv',index_col=0,header=0)
tags_df=pd.read_csv('transition_matrix.csv',index_col=0,header=0)

In [22]:
def viterbi(words,T1=list_of_tags):
    state=[]
    T=list(T1)
    for key,word in enumerate(words):
        if word is not '':
            p=[]
            for tag in T:
                if tag is not '':
                    if key==0:
                        transition_p= prob_of_tag[tag]
                    else:
                        transition_p = tags_df.loc[state[-1], tag]
                    if word in list_of_words:
                        emission_p = emission_df.loc[tag,word]
                        state_probability = emission_p * transition_p    
                    
                    else:
                        emission_p = 0
                        state_probability = emission_p * transition_p 
                    p.append(state_probability)
             
            pmax = max(p)
            state_max = rule_based_tagger.tag([word])[0][1]   
            if(pmax==0):
                state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            else:
                if state_max != 'PUN':
                # getting state for which probability is maximum
                    state_max = T[p.index(pmax)]                
             
         
            state.append(state_max)
    return state
       

In [21]:
patterns = [
    (r'.*ing$', 'VBG'),              # gerund
    (r'.*ed$', 'VVD'),               # past tense 
    (r'.*es$', 'VVZ'),               # verb    
    (r'.*\'s$', 'NN1'),              # possessive nouns
    (r'.*s$', 'NN2'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'PUN'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'CRD'), # cardinal numbers
    (r'.*', 'NN1')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)


In [20]:
import nltk

In [45]:
def process2(dest):
    with open(dest,errors='ignore') as fp:
        for line in fp:
            for item in line.strip().split(' '):
                
                wordtag=item.split('_')
                givenword=wordtag[0]
                giventag=wordtag[-1]
                if givenword is not '':
                    if giventag is not '':
                        actualTags.append(giventag)
                        input1.append(givenword)
                
def process(dest_path):
    for folder in os.listdir(dest_path):
        local_dest=dest_path+'/'+folder
        for file in os.listdir(local_dest):
            process2(local_dest+"/"+file)




actualTags=[]
input1=[]
data1 = ['Test-corpus']
for i in data1:
    #source_path=os.path.join(home,i)
    path=os.path.join(dest_path,i)
   
    process(path)


In [25]:
output=viterbi(input1)

In [50]:
count=0
for i, j in zip(output,actualTags):
    if (i==j):
        count=count+1
    
 
accuracy = count/len(actualTags)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  92.2369088294609


In [59]:
data2 = {'y_Actual':   actualTags ,
        'y_Predicted':output 
        }

df2 = pd.DataFrame(data2, columns=['y_Actual','y_Predicted'])

confusion_matrix2 = pd.crosstab(df2['y_Actual'], df2['y_Predicted'])

confusion_matrix2.to_csv('cmatrixhmm.csv')

In [61]:

confusion_matrix=pd.read_csv('cmatrixhmm.csv',index_col=0,header=0)

confusion_matrix

,AJ0,AJC,AJS,AT0,AV0,AVP,AVQ,CJC,CJS,CJT,...,VHZ,VM0,VVB,VVD,VVG,VVI,VVN,VVZ,XX0,ZZ0
y_Actual,,,,,,,,,,,,,,,,,,,,,
AJ0,251485,0,0,0,3992,174,0,0,0,0,...,0,23,362,5363,2693,152,3986,2,0,8
AJC,0,6809,1,0,924,0,0,0,0,0,...,0,0,5,7,0,27,0,0,0,0
AJS,0,0,3566,0,178,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
AT0,0,0,0,341727,26,0,0,0,0,0,...,0,0,0,0,0,36,0,0,0,193
AV0,5697,1005,163,340,162604,527,14,14,1796,119,...,0,0,16,7,12,33,6,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VVI,242,5,0,8,18,3,0,0,0,0,...,0,2,7062,71,0,77505,187,0,0,0
VVN,3318,0,0,0,10,0,0,0,75,0,...,0,28,355,12053,0,72,81896,0,0,0
VVZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26821,0,0


In [62]:
no_of_times_predicted=[]
F1_score=[]
print("Tag"+'\t\t'+"precision"+'\t\t\t\t'+"recall"+'\t\t\t\t'+"F-score")
for tag in confusion_matrix.index.values.tolist():
    TP=confusion_matrix.at[tag,tag]
    row_list=confusion_matrix.loc[tag,:].values.tolist()
    FN=sum(row_list)-TP
    column_list=confusion_matrix.loc[:,tag].values.tolist()
    FP=sum(column_list)-TP
    if TP+FP==0:
        p=0
    else:
        p=TP/(TP+FP)
    if TP+FN==0:
        r=0
    else:
        r=TP/(TP+FN)
    if p+r==0:
        Fscore=0
    else:
        Fscore = 2 * (p * r) / (p+r)
    no_of_times_predicted.append(sum(column_list))
    F1_score.append(Fscore)
    print(tag+'\t\t'+str(p)+'\t\t'+str(r)+'\t\t'+str(Fscore))

print("Macro F1 score: "+str(sum(F1_score)/len(F1_score)))
num=0
den=0
for (n,f) in zip(no_of_times_predicted,F1_score):
    num+=(n*f)
    den+=n
print("Weighted F1 score: "+str(num/den))

Tag		precision				recall				F-score
AJ0		0.892708814028611		0.8578421339882658		0.8749282446466158
AJC		0.8603740207227698		0.8596136851407651		0.8599936848752763
AJS		0.9542413700829543		0.9352216102806189		0.944635761589404
AT0		0.9956558222470849		0.9975653971117553		0.9966096949600382
AV0		0.9161459712542327		0.889523465664473		0.9026384594461068
AVP		0.8479779512088632		0.7658279320797496		0.8048120378643668
AVQ		0.8511979823455234		0.7744924977934686		0.8110356301122973
CJC		0.9988132645673641		0.9998879233694719		0.9993503050579126
CJS		0.7990066591797029		0.8447580982323528		0.821245671365166
CJT		0.8550232888121163		0.9330169308963614		0.8923190831831059
CRD		0.9484385452240961		0.9438527031288322		0.9461400674346727
DPS		0.9506085312536925		0.9757428744693754		0.9630117309073497
DT0		0.9389337001118986		0.8804472571072565		0.9087504160015343
DTQ		0.9999572777374289		0.999274217649319		0.9996156310057654
EX0		0.8536631419939577		0.954401519949335		0.9012259543506429
ITJ		0.831